In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os


In [2]:
import tensorflow_datasets as tfds

# Load the dataset with splits
(train_ds, val_ds), ds_info = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.QYGOQC_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [3]:
IMG_SIZE = 224
BATCH_SIZE = 32

def format_example(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize
    return image, label

train_ds = train_ds.map(format_example).batch(BATCH_SIZE).prefetch(1)
val_ds = val_ds.map(format_example).batch(BATCH_SIZE).prefetch(1)


In [4]:
base_model = VGG16(input_shape=(224, 224, 3),
                   include_top=False,
                   weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])


In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=5)


Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 144s 219ms/step - accuracy: 0.8340 - loss: 0.6042 - val_accuracy: 0.8960 - val_loss: 0.2253
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 119s 205ms/step - accuracy: 0.9094 - loss: 0.2185 - val_accuracy: 0.9271 - val_loss: 0.1714
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 118s 203ms/step - accuracy: 0.9209 - loss: 0.1916 - val_accuracy: 0.9291 - val_loss: 0.1655
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 118s 203ms/step - accuracy: 0.9329 - loss: 0.1639 - val_accuracy: 0.9308 - val_loss: 0.1610
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 119s 204ms/step - accuracy: 0.9349 - loss: 0.1566 - val_accuracy: 0.9319 - val_loss: 0.1922


In [ ]:
# Unfreeze some layers of the base model
base_model.trainable = True

# Freeze all layers except the last 4 layers of VGG16
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Recompile the model with a lower learning rate
model.compile(optimizer=optimizers.Adam(1e-5),  # Lower LR
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(train_ds,
                              validation_data=val_ds,
                              epochs=5)


In [ ]:
def plot_history(history, fine_tune_history):
    acc = history.history['accuracy'] + fine_tune_history.history['accuracy']
    val_acc = history.history['val_accuracy'] + fine_tune_history.history['val_accuracy']
    loss = history.history['loss'] + fine_tune_history.history['loss']
    val_loss = history.history['val_loss'] + fine_tune_history.history['val_loss']

    epochs_range = range(len(acc))

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy')
    plt.plot(epochs_range, val_acc, label='Val Accuracy')
    plt.legend()
    plt.title('Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Train Loss')
    plt.plot(epochs_range, val_loss, label='Val Loss')
    plt.legend()
    plt.title('Loss')
    plt.show()

plot_history(history, fine_tune_history)


### Explanation of `standard_output` and `standard_error` (referred to as 'outpy' in some contexts)

When a code cell in a notebook environment (like Google Colab) is executed, it can produce two primary types of textual output:

1.  **`standard_output`**: This is the normal output generated by the code, such as results printed using `print()` statements, function return values (if not assigned to a variable and the cell is the last one), or progress messages from libraries. This output typically indicates successful execution and information the user intended to see.

2.  **`standard_error`**: This stream is used for diagnostic output, warnings, or error messages. Code might still complete successfully even if it produces warnings in `standard_error`. If an unhandled exception occurs, the traceback and error message will also appear here.

Monitoring both `standard_output` and `standard_error` is crucial for debugging and understanding the execution flow and potential issues in your code.